In [1]:
# Cell 1: importy i konfiguracja ścieżki do modułów projektu
import json
from pathlib import Path
import sys

import yaml  # type: ignore

# Upewniamy się, że katalog "src" (z modułami projektu) jest na ścieżce importów
src_path = Path("src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from axiomatic_kernel import (
    AxiomKernel,
    VariableSchema,
    AxiomDefinition,
    DecisionLogger,
)
from nl_rule_parser import build_axiom_from_nl, RuleParseError  # noqa: F401
from explanation_engine import DecisionExplainer, ExplanationConfig
from rules_io import load_ruleset_from_file  # noqa: F401
from ruleset_manager import RulesetRegistry, Environment
from rule_analytics import RuleAnalyticsEngine


In [2]:
# Cell: konfiguracja schematu AML, kernela i wczytanie rulesetu AML (governance)


# === 1) SCHEMA DOPASOWANA DO AML_DATA_GENERATOR (tylko pola używane w regułach) ===
schema_aml = [
    VariableSchema(
        "amount",
        "int",
        "Kwota transakcji w jednostkach minimalnych.",
    ),
    VariableSchema(
        "tx_count_24h",
        "int",
        "Liczba transakcji klienta w ostatnich 24h.",
    ),
    VariableSchema(
        "total_amount_24h",
        "int",
        "Łączna kwota transakcji klienta w ostatnich 24h.",
    ),
    VariableSchema(
        "tx_count_7d",
        "int",
        "Liczba transakcji klienta w ostatnich 7 dniach.",
    ),
    VariableSchema(
        "total_amount_7d",
        "int",
        "Łączna kwota transakcji klienta w ostatnich 7 dniach.",
    ),
    VariableSchema(
        "unique_counterparties_30d",
        "int",
        "Liczba unikalnych kontrahentów w ostatnich 30 dniach.",
    ),
    VariableSchema(
        "model_risk_score",
        "real",
        "Wynik modelu ryzyka (0–1).",
    ),
    VariableSchema(
        "is_pep",
        "bool",
        "Czy klient jest PEP.",
    ),
    VariableSchema(
        "on_sanctions_list",
        "bool",
        "Czy klient znajduje się na liście sankcyjnej.",
    ),
    VariableSchema(
        "is_suspicious",
        "bool",
        "Decyzja systemu – czy transakcja jest podejrzana.",
    ),
]


In [3]:
logs_dir = Path("logs")
logs_dir.mkdir(exist_ok=True)

rules_dir = Path("rules")
rules_dir.mkdir(exist_ok=True)
fraud_rules_path = rules_dir / "aml_rules_v1.yaml"

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

In [4]:
analytics_engine = RuleAnalyticsEngine()

analytics_result = analytics_engine.analyze_log_file(
    log_path=logs_dir / "aml_rules_demo.jsonl",
    ruleset_path=fraud_rules_path,
)

report = analytics_result.as_dict()

print("=== STATYSTYKI DECYZJI ===")
print(json.dumps(report["outcome_stats"], indent=2, ensure_ascii=False))

print("\n=== STATYSTYKI REGUŁ ===")
for rule_id, stats in sorted(report["rule_stats"].items()):
    print(f"\nReguła: {rule_id}")
    print(json.dumps(stats, indent=2, ensure_ascii=False))

coverage = report.get("coverage_report")
if coverage is not None:
    print("\n=== POKRYCIE RULESETU ===")
    print(json.dumps(coverage, indent=2, ensure_ascii=False))


=== STATYSTYKI DECYZJI ===
{
  "total_decisions": 8000,
  "by_decision": {
    "FLAGGED": 3620,
    "CLEAN": 4380
  },
  "by_status": {
    "SAT": 8000
  },
  "by_rule_version": {
    "aml_rules_v1:1.0.0@DEV": 8000
  },
  "unsat_cases": 0,
  "error_cases": 0
}

=== STATYSTYKI REGUŁ ===

Reguła: aml.high_amount_core
{
  "rule_id": "aml.high_amount_core",
  "description": "IF amount >= 15000 THEN is_suspicious = TRUE",
  "total_occurrences": 8000,
  "satisfied": 8000,
  "violated": 0,
  "active": 2036,
  "inactive": 5964,
  "in_conflict": 0
}

Reguła: aml.model_score_very_high
{
  "rule_id": "aml.model_score_very_high",
  "description": "IF model_risk_score >= 0.9 THEN is_suspicious = TRUE",
  "total_occurrences": 8000,
  "satisfied": 8000,
  "violated": 0,
  "active": 392,
  "inactive": 7608,
  "in_conflict": 0
}

Reguła: aml.pep_high_risk_amount
{
  "rule_id": "aml.pep_high_risk_amount",
  "description": "IF is_pep == TRUE AND amount >= 2000 THEN is_suspicious = TRUE",
  "total_occurre

In [5]:
# Cell: Rule Insights – definicje (FAZA 4.2)

from __future__ import annotations

from dataclasses import dataclass, field
from typing import Any, Dict, List, Literal, Optional, Tuple

from rule_analytics import (
    DecisionLogReader,
    DecisionOutcomeStats,
    RuleAnalyticsResult,
    RuleStats,
)

LanguageCode = Literal["pl", "en"]


@dataclass
class RuleInsight:
    """Wniosek dotyczący pojedynczej reguły na podstawie statystyk z logów."""

    rule_id: str
    description: Optional[str]
    activity_rate: float
    coverage_rate: float
    conflict_rate: float
    flagged_activation_rate: float
    clean_activation_rate: float
    total_occurrences: int
    active: int
    inactive: int
    in_conflict: int
    classification: str
    insight_text: str


@dataclass
class SystemInsight:
    """Podsumowanie systemu regułowego jako całości."""

    total_decisions: int
    share_flagged: float
    share_clean: float
    share_other: float
    headline: str
    # Najważniejsze (najbardziej aktywne) reguły
    top_rules: List[str] = field(default_factory=list)
    # Reguły rzadko używane / martwe
    low_usage_rules: List[str] = field(default_factory=list)
    # Próg, wg którego liczymy „słabe użycie” (0.05 = 5%)
    low_usage_threshold: float = 0.0


@dataclass
class RuleInsightsReport:
    """Pełny raport wniosków z analizy reguł."""

    system: SystemInsight
    rules: Dict[str, RuleInsight] = field(default_factory=dict)

    def as_dict(self) -> Dict[str, Any]:
        return {
            "system": {
                "total_decisions": self.system.total_decisions,
                "share_flagged": self.system.share_flagged,
                "share_clean": self.system.share_clean,
                "share_other": self.system.share_other,
                "headline": self.system.headline,
                "top_rules": list(self.system.top_rules),
                "low_usage_rules": list(self.system.low_usage_rules),
                "low_usage_threshold": self.system.low_usage_threshold,
            },
            "rules": {
                rule_id: {
                    "rule_id": insight.rule_id,
                    "description": insight.description,
                    "activity_rate": insight.activity_rate,
                    "coverage_rate": insight.coverage_rate,
                    "conflict_rate": insight.conflict_rate,
                    "flagged_activation_rate": insight.flagged_activation_rate,
                    "clean_activation_rate": insight.clean_activation_rate,
                    "total_occurrences": insight.total_occurrences,
                    "active": insight.active,
                    "inactive": insight.inactive,
                    "in_conflict": insight.in_conflict,
                    "classification": insight.classification,
                    "insight_text": insight.insight_text,
                }
                for rule_id, insight in sorted(self.rules.items())
            },
        }


class RuleInsightsEngine:
    """Warstwa „Rule Insights” nad RuleAnalyticsResult.

    Parametry:
        language:
            "pl" lub "en" – język tekstowych insightów.
        high_activity_threshold:
            Próg aktywności, powyżej którego reguła jest klasyfikowana jako "core".
        medium_activity_threshold:
            Próg aktywności dla klasy "supporting".
        min_occurrences_for_insight:
            Minimalna liczba wystąpień, żeby w ogóle klasyfikować regułę.
        low_usage_activity_threshold:
            Próg aktywności, poniżej którego reguła jest traktowana jako
            „słabo używana” (np. 0.05 = 5% decyzji).
    """

    def __init__(
        self,
        *,
        language: LanguageCode = "pl",
        high_activity_threshold: float = 0.3,
        medium_activity_threshold: float = 0.1,
        min_occurrences_for_insight: int = 1,
        low_usage_activity_threshold: float = 0.05,
    ) -> None:
        self._language: LanguageCode = language
        self._high_activity_threshold: float = high_activity_threshold
        self._medium_activity_threshold: float = medium_activity_threshold
        self._min_occurrences_for_insight: int = min_occurrences_for_insight
        # Próg słabego użycia – bezpiecznie obcięty do [0.0, 1.0]
        self._low_usage_activity_threshold: float = max(
            0.0, min(low_usage_activity_threshold, 1.0)
        )

    def build_report(
        self,
        analytics: RuleAnalyticsResult,
        *,
        log_path: Optional[str] = None,
    ) -> RuleInsightsReport:
        """Buduje raport wniosków na podstawie RuleAnalyticsResult."""
        outcome_stats = analytics.outcome_stats
        total_decisions = max(outcome_stats.total_decisions, 1)

        flagged_total = outcome_stats.by_decision.get("FLAGGED", 0)
        clean_total = outcome_stats.by_decision.get("CLEAN", 0)
        other_total = total_decisions - flagged_total - clean_total

        if log_path is not None:
            (
                active_in_flagged,
                active_in_clean,
            ) = self._compute_activation_by_decision_outcome(
                log_path=log_path,
            )
        else:
            active_in_flagged, active_in_clean = {}, {}

        system_insight = self._build_system_insight(
            outcome_stats=outcome_stats,
            total_decisions=total_decisions,
            flagged_total=flagged_total,
            clean_total=clean_total,
            other_total=other_total,
            analytics=analytics,
        )

        rule_insights: Dict[str, RuleInsight] = {}

        for rule_id, stats in analytics.rule_stats.items():
            insight = self._build_rule_insight(
                rule_id=rule_id,
                stats=stats,
                outcome_stats=outcome_stats,
                total_decisions=total_decisions,
                flagged_total=flagged_total,
                clean_total=clean_total,
                active_in_flagged=active_in_flagged.get(rule_id, 0),
                active_in_clean=active_in_clean.get(rule_id, 0),
            )
            rule_insights[rule_id] = insight

        return RuleInsightsReport(system=system_insight, rules=rule_insights)

    def _build_system_insight(
        self,
        *,
        outcome_stats: DecisionOutcomeStats,
        total_decisions: int,
        flagged_total: int,
        clean_total: int,
        other_total: int,
        analytics: RuleAnalyticsResult,
    ) -> SystemInsight:
        share_flagged = flagged_total / total_decisions
        share_clean = clean_total / total_decisions
        share_other = max(other_total, 0) / total_decisions

        if self._language == "pl":
            headline = (
                f"System przetworzył {total_decisions} decyzji. "
                f"{share_clean:.1%} transakcji zostało ocenionych jako CZYSTE, "
                f"{share_flagged:.1%} jako OFLAGOWANE, "
                f"{share_other:.1%} to pozostałe statusy."
            )
        else:
            headline = (
                f"The system processed {total_decisions} decisions. "
                f"{share_clean:.1%} of transactions were marked as CLEAN, "
                f"{share_flagged:.1%} as FLAGGED, "
                f"{share_other:.1%} had other statuses."
            )

        # TOP reguły po liczbie aktywacji
        top_rules_stats = sorted(
            analytics.rule_stats.values(),
            key=lambda r: r.active,
            reverse=True,
        )[:5]

        top_rule_lines: List[str] = []
        for stats in top_rules_stats:
            if total_decisions == 0:
                activity_pct = 0.0
            else:
                activity_pct = stats.active / total_decisions
            if self._language == "pl":
                line = (
                    f"- Reguła '{stats.rule_id}' była aktywna "
                    f"w {activity_pct:.1%} wszystkich decyzji "
                    f"({stats.active}/{total_decisions})."
                )
            else:
                line = (
                    f"- Rule '{stats.rule_id}' was active in "
                    f"{activity_pct:.1%} of all decisions "
                    f"({stats.active}/{total_decisions})."
                )
            top_rule_lines.append(line)

        # Reguły rzadko używane / martwe
        low_usage_lines: List[str] = []

        coverage = analytics.coverage_report
        # 1) Reguły enabled=True, które w ogóle nie pojawiły się w logach
        if coverage is not None and coverage.unused_rules:
            for rule_id in coverage.unused_rules:
                if self._language == "pl":
                    line = (
                        f"- Reguła '{rule_id}' nie pojawiła się w żadnej decyzji "
                        "(0.0% wszystkich decyzji) – kandydatka na martwą lub "
                        "wymagającą dopasowania."
                    )
                else:
                    line = (
                        f"- Rule '{rule_id}' did not appear in any decision "
                        "(0.0% of all decisions) – candidate for deprecation or "
                        "redesign."
                    )
                low_usage_lines.append(line)

        # 2) Reguły, które są w statystykach, ale mają niski poziom aktywności
        if total_decisions > 0:
            for stats in sorted(
                analytics.rule_stats.values(),
                key=lambda r: r.active,
            ):
                activity_pct = stats.active / total_decisions

                # Jeżeli mamy też unused_rules z coverage, unikamy dublowania
                if activity_pct == 0.0:
                    if coverage is not None and stats.rule_id in coverage.unused_rules:
                        continue

                if activity_pct <= self._low_usage_activity_threshold:
                    if self._language == "pl":
                        line = (
                            f"- Reguła '{stats.rule_id}' była aktywna tylko w "
                            f"{activity_pct:.1%} decyzji "
                            f"({stats.active}/{total_decisions})."
                        )
                    else:
                        line = (
                            f"- Rule '{stats.rule_id}' was active in only "
                            f"{activity_pct:.1%} of decisions "
                            f"({stats.active}/{total_decisions})."
                        )
                    low_usage_lines.append(line)

        return SystemInsight(
            total_decisions=total_decisions,
            share_flagged=share_flagged,
            share_clean=share_clean,
            share_other=share_other,
            headline=headline,
            top_rules=top_rule_lines,
            low_usage_rules=low_usage_lines,
            low_usage_threshold=self._low_usage_activity_threshold,
        )

    def _build_rule_insight(
        self,
        *,
        rule_id: str,
        stats: RuleStats,
        outcome_stats: DecisionOutcomeStats,
        total_decisions: int,
        flagged_total: int,
        clean_total: int,
        active_in_flagged: int,
        active_in_clean: int,
    ) -> RuleInsight:
        occurrences = stats.total_occurrences
        active = stats.active
        inactive = stats.inactive
        in_conflict = stats.in_conflict

        activity_rate = active / total_decisions if total_decisions else 0.0
        coverage_rate = (
            occurrences / total_decisions if total_decisions else 0.0
        )
        conflict_rate = (
            in_conflict / total_decisions if total_decisions else 0.0
        )

        flagged_activation_rate = (
            active_in_flagged / flagged_total if flagged_total else 0.0
        )
        clean_activation_rate = (
            active_in_clean / clean_total if clean_total else 0.0
        )

        classification = self._classify_rule(
            occurrences=occurrences,
            activity_rate=activity_rate,
            conflict_rate=conflict_rate,
        )

        insight_text = self._render_rule_insight_text(
            rule_id=rule_id,
            stats=stats,
            total_decisions=total_decisions,
            activity_rate=activity_rate,
            coverage_rate=coverage_rate,
            conflict_rate=conflict_rate,
            flagged_total=flagged_total,
            clean_total=clean_total,
            active_in_flagged=active_in_flagged,
            active_in_clean=active_in_clean,
            flagged_activation_rate=flagged_activation_rate,
            clean_activation_rate=clean_activation_rate,
            classification=classification,
        )

        return RuleInsight(
            rule_id=rule_id,
            description=stats.description,
            activity_rate=activity_rate,
            coverage_rate=coverage_rate,
            conflict_rate=conflict_rate,
            flagged_activation_rate=flagged_activation_rate,
            clean_activation_rate=clean_activation_rate,
            total_occurrences=occurrences,
            active=active,
            inactive=inactive,
            in_conflict=in_conflict,
            classification=classification,
            insight_text=insight_text,
        )

    def _classify_rule(
        self,
        *,
        occurrences: int,
        activity_rate: float,
        conflict_rate: float,
    ) -> str:
        if occurrences < self._min_occurrences_for_insight:
            return "unused"
        if conflict_rate > 0.05:
            return "conflict_prone"
        if activity_rate >= self._high_activity_threshold:
            return "core"
        if activity_rate >= self._medium_activity_threshold:
            return "supporting"
        return "niche"

    def _render_rule_insight_text(
        self,
        *,
        rule_id: str,
        stats: RuleStats,
        total_decisions: int,
        activity_rate: float,
        coverage_rate: float,
        conflict_rate: float,
        flagged_total: int,
        clean_total: int,
        active_in_flagged: int,
        active_in_clean: int,
        flagged_activation_rate: float,
        clean_activation_rate: float,
        classification: str,
    ) -> str:
        total_decisions = max(total_decisions, 1)

        if self._language == "pl":
            parts: List[str] = []

            parts.append(
                f"Reguła '{rule_id}' pojawiła się w "
                f"{coverage_rate:.1%} wszystkich decyzji "
                f"({stats.total_occurrences}/{total_decisions})."
            )
            parts.append(
                f"Aktywnie zadziałała w {activity_rate:.1%} decyzji "
                f"({stats.active}/{total_decisions})."
            )

            if flagged_total > 0 and active_in_flagged > 0:
                parts.append(
                    f"Była aktywna w {flagged_activation_rate:.1%} wszystkich decyzji "
                    f"OFLAGOWANYCH ({active_in_flagged}/{flagged_total})."
                )

            if clean_total > 0 and active_in_clean > 0:
                parts.append(
                    f"Była aktywna w {clean_activation_rate:.1%} wszystkich decyzji "
                    f"CLEAN ({active_in_clean}/{clean_total})."
                )

            if stats.in_conflict > 0:
                parts.append(
                    f"W {stats.in_conflict} decyzjach reguła znalazła się w jądrze UNSAT, "
                    f"co stanowi {conflict_rate:.1%} wszystkich decyzji – kandydat do przeglądu."
                )

            classification_comment = self._classification_comment_pl(
                classification=classification,
            )
            if classification_comment:
                parts.append(classification_comment)

            return " ".join(parts)

        parts_en: List[str] = []

        parts_en.append(
            f"Rule '{rule_id}' appeared in "
            f"{coverage_rate:.1%} of all decisions "
            f"({stats.total_occurrences}/{total_decisions})."
        )
        parts_en.append(
            f"It was actively triggered in {activity_rate:.1%} of decisions "
            f"({stats.active}/{total_decisions})."
        )

        if flagged_total > 0 and active_in_flagged > 0:
            parts_en.append(
                f"It was active in {flagged_activation_rate:.1%} of all FLAGGED decisions "
                f"({active_in_flagged}/{flagged_total})."
            )

        if clean_total > 0 and active_in_clean > 0:
            parts_en.append(
                f"It was active in {clean_activation_rate:.1%} of all CLEAN decisions "
                f"({active_in_clean}/{clean_total})."
            )

        if stats.in_conflict > 0:
            parts_en.append(
                f"In {stats.in_conflict} decisions the rule contributed to UNSAT core "
                f"({conflict_rate:.1%} of all decisions) – candidate for review."
            )

        classification_comment_en = self._classification_comment_en(
            classification=classification,
        )
        if classification_comment_en:
            parts_en.append(classification_comment_en)

        return " ".join(parts_en)

    def _classification_comment_pl(self, classification: str) -> str:
        if classification == "core":
            return (
                "Jest to jedna z kluczowych reguł systemu – wysoka aktywność i "
                "istotny wpływ na decyzje."
            )
        if classification == "supporting":
            return (
                "Reguła wspierająca: używana regularnie, ale nie dominuje w systemie."
            )
        if classification == "niche":
            return (
                "Reguła niszowa: działa rzadko, może opisywać specjalne scenariusze "
                "lub wymaga przeglądu progów."
            )
        if classification == "conflict_prone":
            return (
                "Reguła często uczestniczy w konfliktach (UNSAT) – silna rekomendacja "
                "do przeglądu logicznego."
            )
        if classification == "unused":
            return (
                "Reguła praktycznie nie występuje w logach – kandydatka na martwą lub "
                "wymagającą dopasowania."
            )
        return ""

    def _classification_comment_en(self, classification: str) -> str:
        if classification == "core":
            return (
                "This is one of the core rules – high activity and significant impact "
                "on decisions."
            )
        if classification == "supporting":
            return (
                "Supporting rule: used regularly but does not dominate the system."
            )
        if classification == "niche":
            return (
                "Niche rule: rarely triggers; may describe special scenarios or "
                "require threshold tuning."
            )
        if classification == "conflict_prone":
            return (
                "The rule frequently appears in UNSAT cores – strong candidate for "
                "logical review."
            )
        if classification == "unused":
            return (
                "The rule is practically unused in logs – candidate for deprecation or "
                "redesign."
            )
        return ""

    def _compute_activation_by_decision_outcome(
        self,
        *,
        log_path: str,
    ) -> Tuple[Dict[str, int], Dict[str, int]]:
        active_in_flagged: Dict[str, int] = {}
        active_in_clean: Dict[str, int] = {}

        reader = DecisionLogReader(log_path)

        for record in reader.iter_decisions():
            bundle = record.bundle
            decision = str(bundle.get("decision", "UNKNOWN"))
            active_axioms = bundle.get("active_axioms") or []

            active_ids_in_decision = {
                str(entry.get("id", ""))
                for entry in active_axioms
                if entry.get("id")
            }

            if decision == "FLAGGED":
                target = active_in_flagged
            elif decision == "CLEAN":
                target = active_in_clean
            else:
                target = None

            if target is None:
                continue

            for rule_id in active_ids_in_decision:
                target[rule_id] = target.get(rule_id, 0) + 1

        return active_in_flagged, active_in_clean


In [6]:
# Cell: Rule Insights – definicje (FAZA 4.2)

from __future__ import annotations

from dataclasses import dataclass, field
from typing import Any, Dict, List, Literal, Optional, Tuple

from rule_analytics import (
    DecisionLogReader,
    DecisionOutcomeStats,
    RuleAnalyticsResult,
    RuleStats,
)

LanguageCode = Literal["pl", "en"]


@dataclass
class RuleInsight:
    """Wniosek dotyczący pojedynczej reguły na podstawie statystyk z logów."""

    rule_id: str
    description: Optional[str]
    activity_rate: float
    coverage_rate: float
    conflict_rate: float
    flagged_activation_rate: float
    clean_activation_rate: float
    total_occurrences: int
    active: int
    inactive: int
    in_conflict: int
    classification: str
    insight_text: str


@dataclass
class SystemInsight:
    """Podsumowanie systemu regułowego jako całości."""

    total_decisions: int
    share_flagged: float
    share_clean: float
    share_other: float
    headline: str
    # Najważniejsze (najbardziej aktywne) reguły
    top_rules: List[str] = field(default_factory=list)
    # Reguły rzadko używane / martwe
    low_usage_rules: List[str] = field(default_factory=list)
    # Próg, wg którego liczymy „słabe użycie” (0.05 = 5%)
    low_usage_threshold: float = 0.0


@dataclass
class RuleInsightsReport:
    """Pełny raport wniosków z analizy reguł."""

    system: SystemInsight
    rules: Dict[str, RuleInsight] = field(default_factory=dict)

    def as_dict(self) -> Dict[str, Any]:
        return {
            "system": {
                "total_decisions": self.system.total_decisions,
                "share_flagged": self.system.share_flagged,
                "share_clean": self.system.share_clean,
                "share_other": self.system.share_other,
                "headline": self.system.headline,
                "top_rules": list(self.system.top_rules),
                "low_usage_rules": list(self.system.low_usage_rules),
                "low_usage_threshold": self.system.low_usage_threshold,
            },
            "rules": {
                rule_id: {
                    "rule_id": insight.rule_id,
                    "description": insight.description,
                    "activity_rate": insight.activity_rate,
                    "coverage_rate": insight.coverage_rate,
                    "conflict_rate": insight.conflict_rate,
                    "flagged_activation_rate": insight.flagged_activation_rate,
                    "clean_activation_rate": insight.clean_activation_rate,
                    "total_occurrences": insight.total_occurrences,
                    "active": insight.active,
                    "inactive": insight.inactive,
                    "in_conflict": insight.in_conflict,
                    "classification": insight.classification,
                    "insight_text": insight.insight_text,
                }
                for rule_id, insight in sorted(self.rules.items())
            },
        }


class RuleInsightsEngine:
    """Warstwa „Rule Insights” nad RuleAnalyticsResult.

    Parametry:
        language:
            "pl" lub "en" – język tekstowych insightów.
        high_activity_threshold:
            Próg aktywności, powyżej którego reguła jest klasyfikowana jako "core".
        medium_activity_threshold:
            Próg aktywności dla klasy "supporting".
        min_occurrences_for_insight:
            Minimalna liczba wystąpień, żeby w ogóle klasyfikować regułę.
        low_usage_activity_threshold:
            Próg aktywności, poniżej którego reguła jest traktowana jako
            „słabo używana” (np. 0.05 = 5% decyzji).
    """

    def __init__(
        self,
        *,
        language: LanguageCode = "pl",
        high_activity_threshold: float = 0.3,
        medium_activity_threshold: float = 0.1,
        min_occurrences_for_insight: int = 1,
        low_usage_activity_threshold: float = 0.05,
    ) -> None:
        self._language: LanguageCode = language
        self._high_activity_threshold: float = high_activity_threshold
        self._medium_activity_threshold: float = medium_activity_threshold
        self._min_occurrences_for_insight: int = min_occurrences_for_insight
        # Próg słabego użycia – bezpiecznie obcięty do [0.0, 1.0]
        self._low_usage_activity_threshold: float = max(
            0.0, min(low_usage_activity_threshold, 1.0)
        )

    def build_report(
        self,
        analytics: RuleAnalyticsResult,
        *,
        log_path: Optional[str] = None,
    ) -> RuleInsightsReport:
        """Buduje raport wniosków na podstawie RuleAnalyticsResult."""
        outcome_stats = analytics.outcome_stats
        total_decisions = max(outcome_stats.total_decisions, 1)

        flagged_total = outcome_stats.by_decision.get("FLAGGED", 0)
        clean_total = outcome_stats.by_decision.get("CLEAN", 0)
        other_total = total_decisions - flagged_total - clean_total

        if log_path is not None:
            (
                active_in_flagged,
                active_in_clean,
            ) = self._compute_activation_by_decision_outcome(
                log_path=log_path,
            )
        else:
            active_in_flagged, active_in_clean = {}, {}

        system_insight = self._build_system_insight(
            outcome_stats=outcome_stats,
            total_decisions=total_decisions,
            flagged_total=flagged_total,
            clean_total=clean_total,
            other_total=other_total,
            analytics=analytics,
        )

        rule_insights: Dict[str, RuleInsight] = {}

        for rule_id, stats in analytics.rule_stats.items():
            insight = self._build_rule_insight(
                rule_id=rule_id,
                stats=stats,
                outcome_stats=outcome_stats,
                total_decisions=total_decisions,
                flagged_total=flagged_total,
                clean_total=clean_total,
                active_in_flagged=active_in_flagged.get(rule_id, 0),
                active_in_clean=active_in_clean.get(rule_id, 0),
            )
            rule_insights[rule_id] = insight

        return RuleInsightsReport(system=system_insight, rules=rule_insights)

    def _build_system_insight(
        self,
        *,
        outcome_stats: DecisionOutcomeStats,
        total_decisions: int,
        flagged_total: int,
        clean_total: int,
        other_total: int,
        analytics: RuleAnalyticsResult,
    ) -> SystemInsight:
        share_flagged = flagged_total / total_decisions
        share_clean = clean_total / total_decisions
        share_other = max(other_total, 0) / total_decisions

        if self._language == "pl":
            headline = (
                f"System przetworzył {total_decisions} decyzji. "
                f"{share_clean:.1%} transakcji zostało ocenionych jako CZYSTE, "
                f"{share_flagged:.1%} jako OFLAGOWANE, "
                f"{share_other:.1%} to pozostałe statusy."
            )
        else:
            headline = (
                f"The system processed {total_decisions} decisions. "
                f"{share_clean:.1%} of transactions were marked as CLEAN, "
                f"{share_flagged:.1%} as FLAGGED, "
                f"{share_other:.1%} had other statuses."
            )

        # TOP reguły po liczbie aktywacji
        top_rules_stats = sorted(
            analytics.rule_stats.values(),
            key=lambda r: r.active,
            reverse=True,
        )[:5]

        top_rule_lines: List[str] = []
        for stats in top_rules_stats:
            if total_decisions == 0:
                activity_pct = 0.0
            else:
                activity_pct = stats.active / total_decisions
            if self._language == "pl":
                line = (
                    f"- Reguła '{stats.rule_id}' była aktywna "
                    f"w {activity_pct:.1%} wszystkich decyzji "
                    f"({stats.active}/{total_decisions})."
                )
            else:
                line = (
                    f"- Rule '{stats.rule_id}' was active in "
                    f"{activity_pct:.1%} of all decisions "
                    f"({stats.active}/{total_decisions})."
                )
            top_rule_lines.append(line)

        # Reguły rzadko używane / martwe
        low_usage_lines: List[str] = []

        coverage = analytics.coverage_report
        # 1) Reguły enabled=True, które w ogóle nie pojawiły się w logach
        if coverage is not None and coverage.unused_rules:
            for rule_id in coverage.unused_rules:
                if self._language == "pl":
                    line = (
                        f"- Reguła '{rule_id}' nie pojawiła się w żadnej decyzji "
                        "(0.0% wszystkich decyzji) – kandydatka na martwą lub "
                        "wymagającą dopasowania."
                    )
                else:
                    line = (
                        f"- Rule '{rule_id}' did not appear in any decision "
                        "(0.0% of all decisions) – candidate for deprecation or "
                        "redesign."
                    )
                low_usage_lines.append(line)

        # 2) Reguły, które są w statystykach, ale mają niski poziom aktywności
        if total_decisions > 0:
            for stats in sorted(
                analytics.rule_stats.values(),
                key=lambda r: r.active,
            ):
                activity_pct = stats.active / total_decisions

                # Jeżeli mamy też unused_rules z coverage, unikamy dublowania
                if activity_pct == 0.0:
                    if coverage is not None and stats.rule_id in coverage.unused_rules:
                        continue

                if activity_pct <= self._low_usage_activity_threshold:
                    if self._language == "pl":
                        line = (
                            f"- Reguła '{stats.rule_id}' była aktywna tylko w "
                            f"{activity_pct:.1%} decyzji "
                            f"({stats.active}/{total_decisions})."
                        )
                    else:
                        line = (
                            f"- Rule '{stats.rule_id}' was active in only "
                            f"{activity_pct:.1%} of decisions "
                            f"({stats.active}/{total_decisions})."
                        )
                    low_usage_lines.append(line)

        return SystemInsight(
            total_decisions=total_decisions,
            share_flagged=share_flagged,
            share_clean=share_clean,
            share_other=share_other,
            headline=headline,
            top_rules=top_rule_lines,
            low_usage_rules=low_usage_lines,
            low_usage_threshold=self._low_usage_activity_threshold,
        )

    def _build_rule_insight(
        self,
        *,
        rule_id: str,
        stats: RuleStats,
        outcome_stats: DecisionOutcomeStats,
        total_decisions: int,
        flagged_total: int,
        clean_total: int,
        active_in_flagged: int,
        active_in_clean: int,
    ) -> RuleInsight:
        occurrences = stats.total_occurrences
        active = stats.active
        inactive = stats.inactive
        in_conflict = stats.in_conflict

        activity_rate = active / total_decisions if total_decisions else 0.0
        coverage_rate = (
            occurrences / total_decisions if total_decisions else 0.0
        )
        conflict_rate = (
            in_conflict / total_decisions if total_decisions else 0.0
        )

        flagged_activation_rate = (
            active_in_flagged / flagged_total if flagged_total else 0.0
        )
        clean_activation_rate = (
            active_in_clean / clean_total if clean_total else 0.0
        )

        classification = self._classify_rule(
            occurrences=occurrences,
            activity_rate=activity_rate,
            conflict_rate=conflict_rate,
        )

        insight_text = self._render_rule_insight_text(
            rule_id=rule_id,
            stats=stats,
            total_decisions=total_decisions,
            activity_rate=activity_rate,
            coverage_rate=coverage_rate,
            conflict_rate=conflict_rate,
            flagged_total=flagged_total,
            clean_total=clean_total,
            active_in_flagged=active_in_flagged,
            active_in_clean=active_in_clean,
            flagged_activation_rate=flagged_activation_rate,
            clean_activation_rate=clean_activation_rate,
            classification=classification,
        )

        return RuleInsight(
            rule_id=rule_id,
            description=stats.description,
            activity_rate=activity_rate,
            coverage_rate=coverage_rate,
            conflict_rate=conflict_rate,
            flagged_activation_rate=flagged_activation_rate,
            clean_activation_rate=clean_activation_rate,
            total_occurrences=occurrences,
            active=active,
            inactive=inactive,
            in_conflict=in_conflict,
            classification=classification,
            insight_text=insight_text,
        )

    def _classify_rule(
        self,
        *,
        occurrences: int,
        activity_rate: float,
        conflict_rate: float,
    ) -> str:
        if occurrences < self._min_occurrences_for_insight:
            return "unused"
        if conflict_rate > 0.05:
            return "conflict_prone"
        if activity_rate >= self._high_activity_threshold:
            return "core"
        if activity_rate >= self._medium_activity_threshold:
            return "supporting"
        return "niche"

    def _render_rule_insight_text(
        self,
        *,
        rule_id: str,
        stats: RuleStats,
        total_decisions: int,
        activity_rate: float,
        coverage_rate: float,
        conflict_rate: float,
        flagged_total: int,
        clean_total: int,
        active_in_flagged: int,
        active_in_clean: int,
        flagged_activation_rate: float,
        clean_activation_rate: float,
        classification: str,
    ) -> str:
        total_decisions = max(total_decisions, 1)

        if self._language == "pl":
            parts: List[str] = []

            parts.append(
                f"Reguła '{rule_id}' pojawiła się w "
                f"{coverage_rate:.1%} wszystkich decyzji "
                f"({stats.total_occurrences}/{total_decisions})."
            )
            parts.append(
                f"Aktywnie zadziałała w {activity_rate:.1%} decyzji "
                f"({stats.active}/{total_decisions})."
            )

            if flagged_total > 0 and active_in_flagged > 0:
                parts.append(
                    f"Była aktywna w {flagged_activation_rate:.1%} wszystkich decyzji "
                    f"OFLAGOWANYCH ({active_in_flagged}/{flagged_total})."
                )

            if clean_total > 0 and active_in_clean > 0:
                parts.append(
                    f"Była aktywna w {clean_activation_rate:.1%} wszystkich decyzji "
                    f"CLEAN ({active_in_clean}/{clean_total})."
                )

            if stats.in_conflict > 0:
                parts.append(
                    f"W {stats.in_conflict} decyzjach reguła znalazła się w jądrze UNSAT, "
                    f"co stanowi {conflict_rate:.1%} wszystkich decyzji – kandydat do przeglądu."
                )

            classification_comment = self._classification_comment_pl(
                classification=classification,
            )
            if classification_comment:
                parts.append(classification_comment)

            return " ".join(parts)

        parts_en: List[str] = []

        parts_en.append(
            f"Rule '{rule_id}' appeared in "
            f"{coverage_rate:.1%} of all decisions "
            f"({stats.total_occurrences}/{total_decisions})."
        )
        parts_en.append(
            f"It was actively triggered in {activity_rate:.1%} of decisions "
            f"({stats.active}/{total_decisions})."
        )

        if flagged_total > 0 and active_in_flagged > 0:
            parts_en.append(
                f"It was active in {flagged_activation_rate:.1%} of all FLAGGED decisions "
                f"({active_in_flagged}/{flagged_total})."
            )

        if clean_total > 0 and active_in_clean > 0:
            parts_en.append(
                f"It was active in {clean_activation_rate:.1%} of all CLEAN decisions "
                f"({active_in_clean}/{clean_total})."
            )

        if stats.in_conflict > 0:
            parts_en.append(
                f"In {stats.in_conflict} decisions the rule contributed to UNSAT core "
                f"({conflict_rate:.1%} of all decisions) – candidate for review."
            )

        classification_comment_en = self._classification_comment_en(
            classification=classification,
        )
        if classification_comment_en:
            parts_en.append(classification_comment_en)

        return " ".join(parts_en)

    def _classification_comment_pl(self, classification: str) -> str:
        if classification == "core":
            return (
                "Jest to jedna z kluczowych reguł systemu – wysoka aktywność i "
                "istotny wpływ na decyzje."
            )
        if classification == "supporting":
            return (
                "Reguła wspierająca: używana regularnie, ale nie dominuje w systemie."
            )
        if classification == "niche":
            return (
                "Reguła niszowa: działa rzadko, może opisywać specjalne scenariusze "
                "lub wymaga przeglądu progów."
            )
        if classification == "conflict_prone":
            return (
                "Reguła często uczestniczy w konfliktach (UNSAT) – silna rekomendacja "
                "do przeglądu logicznego."
            )
        if classification == "unused":
            return (
                "Reguła praktycznie nie występuje w logach – kandydatka na martwą lub "
                "wymagającą dopasowania."
            )
        return ""

    def _classification_comment_en(self, classification: str) -> str:
        if classification == "core":
            return (
                "This is one of the core rules – high activity and significant impact "
                "on decisions."
            )
        if classification == "supporting":
            return (
                "Supporting rule: used regularly but does not dominate the system."
            )
        if classification == "niche":
            return (
                "Niche rule: rarely triggers; may describe special scenarios or "
                "require threshold tuning."
            )
        if classification == "conflict_prone":
            return (
                "The rule frequently appears in UNSAT cores – strong candidate for "
                "logical review."
            )
        if classification == "unused":
            return (
                "The rule is practically unused in logs – candidate for deprecation or "
                "redesign."
            )
        return ""

    def _compute_activation_by_decision_outcome(
        self,
        *,
        log_path: str,
    ) -> Tuple[Dict[str, int], Dict[str, int]]:
        active_in_flagged: Dict[str, int] = {}
        active_in_clean: Dict[str, int] = {}

        reader = DecisionLogReader(log_path)

        for record in reader.iter_decisions():
            bundle = record.bundle
            decision = str(bundle.get("decision", "UNKNOWN"))
            active_axioms = bundle.get("active_axioms") or []

            active_ids_in_decision = {
                str(entry.get("id", ""))
                for entry in active_axioms
                if entry.get("id")
            }

            if decision == "FLAGGED":
                target = active_in_flagged
            elif decision == "CLEAN":
                target = active_in_clean
            else:
                target = None

            if target is None:
                continue

            for rule_id in active_ids_in_decision:
                target[rule_id] = target.get(rule_id, 0) + 1

        return active_in_flagged, active_in_clean


In [7]:
# Cell: Rule Insights – wnioski z FAZY 4.2 na bazie analytics_result

# Próg słabego użycia (np. 0.05 = 5% decyzji)
insights_engine = RuleInsightsEngine(
    language="pl",
    low_usage_activity_threshold=0.05,
)

insights_report = insights_engine.build_report(
    analytics_result,
    log_path=str(logs_dir / "aml_rules_demo.jsonl"),
)

print("=== INSIGHT: PODSUMOWANIE SYSTEMU ===")
print(insights_report.system.headline)
print()

print("Najważniejsze reguły:")
for line in insights_report.system.top_rules:
    print(line)

if insights_report.system.low_usage_rules:
    threshold_pct = insights_report.system.low_usage_threshold * 100.0
    print(
        f"\nReguły rzadko używane (aktywność ≤ {threshold_pct:.1f}% decyzji):"
    )
    for line in insights_report.system.low_usage_rules:
        print(line)

print("\n=== INSIGHT: WNIOSKI PER REGUŁA ===")
for rule_id, insight in sorted(insights_report.rules.items()):
    print(f"\n--- {rule_id} ---")
    print(insight.insight_text)


=== INSIGHT: PODSUMOWANIE SYSTEMU ===
System przetworzył 8000 decyzji. 54.8% transakcji zostało ocenionych jako CZYSTE, 45.2% jako OFLAGOWANE, 0.0% to pozostałe statusy.

Najważniejsze reguły:
- Reguła 'aml.high_amount_core' była aktywna w 25.4% wszystkich decyzji (2036/8000).
- Reguła 'aml.velocity_24h_core' była aktywna w 18.2% wszystkich decyzji (1456/8000).
- Reguła 'aml.velocity_7d_heavy' była aktywna w 13.3% wszystkich decyzji (1064/8000).
- Reguła 'aml.pep_high_risk_amount' była aktywna w 10.9% wszystkich decyzji (876/8000).
- Reguła 'aml.structuring_cash_like' była aktywna w 5.3% wszystkich decyzji (428/8000).

Reguły rzadko używane (aktywność ≤ 5.0% decyzji):
- Reguła 'aml.sanctions_match' była aktywna tylko w 0.0% decyzji (0/8000).
- Reguła 'aml.model_score_very_high' była aktywna tylko w 4.9% decyzji (392/8000).

=== INSIGHT: WNIOSKI PER REGUŁA ===

--- aml.high_amount_core ---
Reguła 'aml.high_amount_core' pojawiła się w 100.0% wszystkich decyzji (8000/8000). Aktywnie zadzi

In [8]:
# Cell: Rule Gaps – analiza segmentów transakcji (FAZA 4.3)

from rule_gaps import RuleGapsConfig, RuleGapsEngine

csv_path = data_dir / "transactions_aml_with_explanations.csv"

gaps_engine = RuleGapsEngine(
    RuleGapsConfig(
        min_total_cases=20,       # minimalna liczba case'ów w segmencie
        min_flagged_rate=0.3,     # min. 30% FLAGGED, żeby segment był "gorący"
        max_distinct_rules_in_gap=2,    # max 1-2 reguły w oflagowanych
        min_dominant_rule_share=0.6,    # dominująca reguła >= 60% FLAGGED
    )
)

gaps_result = gaps_engine.analyze_csv(csv_path)

print(f"📊 Rule Gaps – analiza segmentów na podstawie: {csv_path}")
print(f"  Łączna liczba segmentów: {len(gaps_result.segments)}")
print(f"  Segmenty zidentyfikowane jako potencjalne luki: {len(gaps_result.gap_segments)}\n")

for idx, gap in enumerate(gaps_result.gap_segments, start=1):
    print("=" * 70)
    print(f"🧩 GAP #{idx}: {gap.key.label()}")
    print(
        f"Liczba decyzji w segmencie: {gap.total} "
        f"(FLAGGED: {gap.flagged}, CLEAN: {gap.clean}, "
        f"FLAGGED %: {gap.flagged_rate:.1%})"
    )

    if gap.dominant_rules:
        rules_list = ", ".join(gap.dominant_rules)
        print(f"Dominujące reguły w oflagowanych decyzjach: {rules_list}")
        print(
            "Udział najsilniejszej reguły we FLAGGED: "
            f"{gap.dominant_rules_share:.1%}"
        )
    else:
        print("W oflagowanych decyzjach nie aktywowała się żadna reguła.")

    print("\nWniosek:")
    print(gap.note_text)
    print()

if not gaps_result.gap_segments:
    print("✅ Brak oczywistych luk – żaden segment nie spełnia kryteriów GAP.")


📊 Rule Gaps – analiza segmentów na podstawie: data/transactions_aml_with_explanations.csv
  Łączna liczba segmentów: 32
  Segmenty zidentyfikowane jako potencjalne luki: 2

🧩 GAP #1: amount=[5k, 20k), tx_count_24h=11–20, non-PEP
Liczba decyzji w segmencie: 25 (FLAGGED: 25, CLEAN: 0, FLAGGED %: 100.0%)
Dominujące reguły w oflagowanych decyzjach: aml.velocity_24h_core, aml.velocity_7d_heavy
Udział najsilniejszej reguły we FLAGGED: 100.0%

Wniosek:
Segment amount=[5k, 20k), tx_count_24h=11–20, non-PEP ma 25 decyzji, z czego 25 (100.0%) jest OFLAGOWANYCH. W oflagowanych decyzjach aktywuje się bardzo ograniczony zestaw reguł (łącznie 2), z dominującymi: aml.velocity_24h_core (25 razy), aml.velocity_7d_heavy (12 razy). Najmocniejsza reguła pokrywa około 100.0% wszystkich flagowań w tym segmencie. To sugeruje, że warto rozważyć doprecyzowanie logiki dla tego segmentu (np. rozbicie na bardziej szczegółowe reguły) lub sprawdzenie, czy brak dodatkowych reguł nie powoduje nadmiernego obciążenia p

In [9]:
# Cell: Candidate Rule Engine – generowanie propozycji reguł (FAZA 5)

from src.candidate_rule_engine import (
    CandidateRuleConfig,
    CandidateRuleEngine,
)

# CSV z decyzjami + wyjaśnieniami (wynik batch scoringu)
csv_candidates_path = data_dir / "transactions_aml_with_explanations.csv"

candidate_config = CandidateRuleConfig(
    min_triggered_cases=10,  # możesz podnieść np. do 20 w realnym banku
)

candidate_engine = CandidateRuleEngine(
    schema=schema_aml,
    decision_field="is_suspicious",
    ruleset_path=fraud_rules_path,
    config=candidate_config,
)

candidates = candidate_engine.generate_candidates_from_gaps(
    gaps_result,
    csv_path=csv_candidates_path,
)

print(
    f"📌 Wygenerowano {len(candidates)} kandydatów reguł "
    f"na podstawie {len(gaps_result.gap_segments)} segmentów GAP."
)

for idx, candidate in enumerate(candidates, start=1):
    metrics = candidate.metrics
    proof = candidate.proof

    print("\n" + "=" * 70)
    print(f"🧪 Kandydat #{idx}: {candidate.rule_id}")
    print(f"Segment: {candidate.segment.key.label()}")
    print(f"Reguła (NL): {candidate.nl_rule_text}")
    print(f"Opis: {candidate.description}\n")

    print(
        f"- Transakcje w segmencie (z Rule Gaps): "
        f"{metrics.segment_total} (FLAGGED: {metrics.segment_flagged}, "
        f"CLEAN: {metrics.segment_clean}, "
        f"FLAGGED%: {metrics.segment_flagged_rate:.1%})"
    )
    print(
        f"- Transakcje spełniające warunek kandydata w całym zbiorze: "
        f"{metrics.triggered_total} / {metrics.total_cases} "
        f"({metrics.triggered_share:.1%})"
    )
    print(
        f"- W obszarze działania reguły: "
        f"FLAGGED={metrics.triggered_flagged}, "
        f"CLEAN={metrics.triggered_clean}, "
        f"inne={metrics.triggered_other}"
    )

    if proof.is_conflict_free:
        print("- Spójność z istniejącym rulesetem: ✅ brak konfliktów w Z3")
    else:
        print(
            f"- Spójność z istniejącym rulesetem: ⚠️ {proof.conflict_count} "
            "potencjalnych konfliktów z istniejącymi regułami:"
        )
        for detail in proof.conflict_details:
            print(f"  • {detail}")


📌 Wygenerowano 2 kandydatów reguł na podstawie 2 segmentów GAP.

🧪 Kandydat #1: fraud.candidate_gap.001
Segment: amount=[5k, 20k), tx_count_24h=11–20, non-PEP
Reguła (NL): IF amount >= 5000 AND amount < 20000 AND tx_count_24h >= 11 AND tx_count_24h <= 20 AND is_pep == FALSE THEN is_suspicious = TRUE
Opis: Kandydat reguły dla segmentu: amount=[5k, 20k), tx_count_24h=11–20, non-PEP (automatycznie wygenerowany na podstawie Rule Gaps).

- Transakcje w segmencie (z Rule Gaps): 25 (FLAGGED: 25, CLEAN: 0, FLAGGED%: 100.0%)
- Transakcje spełniające warunek kandydata w całym zbiorze: 25 / 2000 (1.2%)
- W obszarze działania reguły: FLAGGED=25, CLEAN=0, inne=0
- Spójność z istniejącym rulesetem: ✅ brak konfliktów w Z3

🧪 Kandydat #2: fraud.candidate_gap.002
Segment: amount=100k+, tx_count_24h=0–1, non-PEP
Reguła (NL): IF amount >= 100000 AND tx_count_24h >= 0 AND tx_count_24h <= 1 AND is_pep == FALSE THEN is_suspicious = TRUE
Opis: Kandydat reguły dla segmentu: amount=100k+, tx_count_24h=0–1, non

In [10]:
# Cell: Candidate promotion – promowanie reguł do fraud_rules_v2 + porównanie v1 vs v2

from candidate_promotion import (
    PromotionConfig,
    merge_ruleset_with_candidates,
    save_ruleset_to_yaml,
    load_base_ruleset,
)

from ruleset_manager import RulesetRegistry, Environment
from axiomatic_kernel import AxiomKernel, DecisionLogger
from rule_analytics import RuleAnalyticsEngine

# 1) Wyświetlamy dostępnych kandydatów
if "candidates" not in globals() or not candidates:
    raise RuntimeError(
        "Brak listy 'candidates'. Najpierw uruchom komórkę z CandidateRuleEngine."
    )

print("=== DOSTĘPNI KANDYDACI REGUŁ (FAZA 5) ===")
for cand in candidates:
    segment_label = cand.segment.key.label()
    metrics = cand.metrics
    print()
    print(f"- {cand.rule_id}")
    print(f"  Segment: {segment_label}")
    print(f"  NL rule: {cand.nl_rule_text}")
    print(
        f"  Pokrycie: {metrics.triggered_total}/{metrics.total_cases} "
        f"({metrics.triggered_share:.1%} wszystkich case'ów)"
    )
    print(
        f"  W segmencie: total={metrics.segment_total}, "
        f"FLAGGED={metrics.segment_flagged}, CLEAN={metrics.segment_clean} "
        f"(FLAGGED%={metrics.segment_flagged_rate:.1%})"
    )
    print(
        f"  Konflikty w Z3: "
        f"{'brak' if cand.proof.is_conflict_free else f'{cand.proof.conflict_count}'}"
    )

# 2) Wybór kandydatów do promocji (edytowalne przez analityka)
#    Domyślnie bierzemy wszystkich – możesz ręcznie zawęzić tę listę.
accepted_ids = [cand.rule_id for cand in candidates]

print("\nZaakceptowane do promocji (możesz edytować listę accepted_ids):")
for rid in accepted_ids:
    print(f"  - {rid}")

# 3) Budujemy nowy ruleset v2

base_ruleset = load_base_ruleset(fraud_rules_path)

promotion_cfg = PromotionConfig(
    bump_part="minor",
    default_severity="MEDIUM",
    base_tags=["candidate_from_gap", "experimental"],
)

new_ruleset_id = "aml_rules_v2"
new_ruleset = merge_ruleset_with_candidates(
    base_ruleset=base_ruleset,
    candidates=candidates,
    accepted_ids=accepted_ids,
    new_ruleset_id=new_ruleset_id,
    promotion_config=promotion_cfg,
)

new_rules_path = rules_dir / "aml_rules_v2.yaml"
save_ruleset_to_yaml(new_ruleset, new_rules_path, overwrite=True)

print(
    "\n💾 Zapisano nowy ruleset:"
    f" id={new_ruleset.ruleset_id}, version={new_ruleset.version},"
    f" file={new_rules_path}"
)

# 4) Porównanie zachowania v1 vs v2 na tym samym zbiorze transakcji

# Najpierw spróbujmy na transactions_demo.csv; jeśli masz większy zbiór z
# wcześniejszych kroków, możesz podmienić ścieżkę lub użyjemy fallbacku.
comparison_source_path = data_dir / "transactions_aml_compare_dataset.csv"
if not comparison_source_path.exists():
    # Fallback: jeśli istnieje pełny plik z wyjaśnieniami, użyjemy jego.
    alt_path = data_dir / "transactions_aml_with_explanations.csv"
    if alt_path.exists():
        comparison_source_path = alt_path
    else:
        raise FileNotFoundError(
            "Nie znaleziono pliku z danymi do porównania "
            "(transactions_demo.csv ani transactions_with_explanations.csv)."
        )

print(f"\n📊 Porównanie rulesetu v1 vs v2 na danych: {comparison_source_path}")

logs_v1_path = logs_dir / "aml_rules_v1_compare.jsonl"
logs_v2_path = logs_dir / "aml_rules_v2_compare.jsonl"

logger_v1 = DecisionLogger(logs_v1_path)
logger_v2 = DecisionLogger(logs_v2_path)

kernel_v1 = AxiomKernel(
    schema=schema_aml,
    decision_variable="is_suspicious",
    logger=logger_v1,
    rule_version="aml_rules_v1_compare",
)

kernel_v2 = AxiomKernel(
    schema=schema_aml,
    decision_variable="is_suspicious",
    logger=logger_v2,
    rule_version="aml_rules_v2_compare",
)

local_registry = RulesetRegistry()
# Rejestrujemy oba rulesety w DEV
local_registry.register_ruleset(
    ruleset_id="aml_rules_v1",
    path=fraud_rules_path,
    environment=Environment.DEV,
)
local_registry.register_ruleset(
    ruleset_id=new_ruleset_id,
    path=new_rules_path,
    environment=Environment.DEV,
)

summary_v1 = local_registry.apply_ruleset_to_kernel(
    ruleset_id="aml_rules_v1",
    environment=Environment.DEV,
    kernel=kernel_v1,
    schema=schema_aml,
    decision_field_fallback="is_suspicious",
)

summary_v2 = local_registry.apply_ruleset_to_kernel(
    ruleset_id=new_ruleset_id,
    environment=Environment.DEV,
    kernel=kernel_v2,
    schema=schema_aml,
    decision_field_fallback="is_suspicious",
)

print("\n📥 Nałożono rulesety na kernele:")
print(
    f"  - v1: {summary_v1.loaded_rules}/{summary_v1.total_rules} reguł załadowanych"
)
print(
    f"  - v2: {summary_v2.loaded_rules}/{summary_v2.total_rules} reguł załadowanych"
)

import csv as _csv

with comparison_source_path.open("r", newline="", encoding="utf-8") as _f:
    reader = _csv.DictReader(_f)
    rows = list(reader)

for row in rows:
    case = {
        "amount": int(row["amount"]),
        "tx_count_24h": int(row["tx_count_24h"]),
        "is_pep": str(row["is_pep"]).lower()
        in {"true", "1", "yes", "y", "t"},
    }
    kernel_v1.evaluate(case)
    kernel_v2.evaluate(case)

analytics_engine = RuleAnalyticsEngine()

analytics_v1 = analytics_engine.analyze_log_file(
    log_path=logs_v1_path,
    ruleset_path=fraud_rules_path,
)

analytics_v2 = analytics_engine.analyze_log_file(
    log_path=logs_v2_path,
    ruleset_path=new_rules_path,
)

report_v1 = analytics_v1.as_dict()
report_v2 = analytics_v2.as_dict()

def _print_outcome_summary(label: str, report: dict) -> None:
    stats = report["outcome_stats"]
    total = stats["total_decisions"]
    flagged = stats["by_decision"].get("FLAGGED", 0)
    clean = stats["by_decision"].get("CLEAN", 0)
    other = total - flagged - clean
    print(f"\n=== {label} ===")
    print(f"Łączna liczba decyzji: {total}")
    if total > 0:
        print(f"FLAGGED: {flagged} ({flagged/total:.1%})")
        print(f"CLEAN:   {clean} ({clean/total:.1%})")
        if other:
            print(f"INNE:    {other} ({other/total:.1%})")

_print_outcome_summary("FRAUD_RULES_V1", report_v1)
_print_outcome_summary("FRAUD_RULES_V2", report_v2)

print("\n=== Nowe reguły w FRAUD_RULES_V2 ===")
for rid in accepted_ids:
    stats_v2 = report_v2["rule_stats"].get(rid)
    if not stats_v2:
        print(f"- {rid}: brak statystyk (reguła nie wystąpiła w logach).")
        continue
    active = stats_v2["active"]
    total_occ = stats_v2["total_occurrences"]
    print(
        f"- {rid}: aktywna {active} razy, "
        f"obecna w {total_occ} decyzjach."
    )

print(
    "\n✅ Porównanie zakończone. "
    f"Logi zapisano w: {logs_v1_path} (v1), {logs_v2_path} (v2)."
)


=== DOSTĘPNI KANDYDACI REGUŁ (FAZA 5) ===

- fraud.candidate_gap.001
  Segment: amount=[5k, 20k), tx_count_24h=11–20, non-PEP
  NL rule: IF amount >= 5000 AND amount < 20000 AND tx_count_24h >= 11 AND tx_count_24h <= 20 AND is_pep == FALSE THEN is_suspicious = TRUE
  Pokrycie: 25/2000 (1.2% wszystkich case'ów)
  W segmencie: total=25, FLAGGED=25, CLEAN=0 (FLAGGED%=100.0%)
  Konflikty w Z3: brak

- fraud.candidate_gap.002
  Segment: amount=100k+, tx_count_24h=0–1, non-PEP
  NL rule: IF amount >= 100000 AND tx_count_24h >= 0 AND tx_count_24h <= 1 AND is_pep == FALSE THEN is_suspicious = TRUE
  Pokrycie: 22/2000 (1.1% wszystkich case'ów)
  W segmencie: total=22, FLAGGED=22, CLEAN=0 (FLAGGED%=100.0%)
  Konflikty w Z3: brak

Zaakceptowane do promocji (możesz edytować listę accepted_ids):
  - fraud.candidate_gap.001
  - fraud.candidate_gap.002

💾 Zapisano nowy ruleset: id=aml_rules_v2, version=1.1.0, file=rules/aml_rules_v2.yaml

📊 Porównanie rulesetu v1 vs v2 na danych: data/transactions_am